In [33]:
import pandas as pd
import os
import concurrent.futures
import openai
d = pd.read_csv("NEWS_YAHOO_stock_prediction.csv")


In [34]:
d.drop(columns="Unnamed: 0",inplace=True)
d.set_index("Date")


,ticker,category,title,content,Open,High,Low,Close,Adj Close,Volume,label
Date,,,,,,,,,,,
2020-01-27,AAPL,opinion,Apple Set To Beat Q1 Earnings Estimates Tech ...,Technology giant Apple NASDAQ AAPL is set ...,77.514999,77.942497,76.220001,77.237503,75.793358,161940000,0
2020-01-27,AAPL,opinion,Tech Daily Intel Results Netflix Surge Appl...,The top stories in this digest are Intel s N...,77.514999,77.942497,76.220001,77.237503,75.793358,161940000,0
2020-01-27,AAPL,opinion,7 Monster Stock Market Predictions For The Wee...,S P 500 SPY \nThis week will be packed with e...,77.514999,77.942497,76.220001,77.237503,75.793358,161940000,0
2020-01-27,AAPL,opinion,Apple Earnings Preview 5G Launch Expanding S...,Reports Q1 2020 results on Tuesday Jan 28 ...,77.514999,77.942497,76.220001,77.237503,75.793358,161940000,0
2020-01-27,AAPL,opinion,Buy Surging Apple Microsoft Stock Before Qua...,On today s episode of Full Court Finance here ...,77.514999,77.942497,76.220001,77.237503,75.793358,161940000,0
...,...,...,...,...,...,...,...,...,...,...,...
2012-07-31,AAPL,opinion,What s Driving China s Real Estate Rally Par...,In the preceding posts I examined the first t...,21.543928,21.846430,21.525715,21.812857,18.624512,462327600,1
2012-07-31,AAPL,opinion,50 Stocks With The Best Dividend Growth In Jul...,Here is a current sheet of companies with fast...,21.543928,21.846430,21.525715,21.812857,18.624512,462327600,1
2012-07-23,AAPL,opinion,Summer Heat Scorches Europe And U S,Europe flares as summer heat continues Summer...,21.228571,21.639286,20.989643,21.565357,18.413187,487975600,1


In [31]:
from openai import OpenAI

# Set your API key
with open("api.txt", "r") as file:
    api_key = file.read().strip()

# Set the API key
openai.api_key = api_key

# Example API call
response = openai.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a very concise Citadel Analyst"},
        {"role": "user", "content": "How can I use OpenAI in my project?"}
    ],
    max_tokens=150
)

print(response.choices[0].message.content)


To use OpenAI in your project, you can follow these general steps:

1. Get API access: Sign up for OpenAI API access on the OpenAI website. You may need to request access and provide information about your project.

2. Choose the API: OpenAI offers different APIs for different purposes, such as text generation, language translation, image recognition, etc. Choose the API that best suits your project.

3. Set up the API: Follow the documentation provided by OpenAI to set up the API in your project. This may involve installing libraries, setting up authentication, and making API calls.

4. Integrate the API: Write code in your project to make API calls and process the responses. You can use the API to generate


In [38]:
import time


In [37]:
def get_scores(row):
    """
    Function to get sentiment and importance scores for a row of content and title.
    """
    try:
        # Prepare the prompt
        prompt = f"""
        Analyze the following article:
        Title: {row['title']}
        Content: {row['content']}
        
        Provide two scores:
        1. Sentiment score (scale -1 to 1): How positive or negative is the sentiment of the article?
        2. Importance score (scale 0 to 1): How important is this article's content for its context?
        
        Format your response as:
        Sentiment: <sentiment_score>
        Importance: <importance_score>
        """
        
        # Call the OpenAI API
        response = openai.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=100,
            temperature=0.7
        )
        
        # Extract the scores from the response
        reply = response.choices[0].message.content
        sentiment_score = float(reply.split("Sentiment:")[1].split("\n")[0].strip())
        importance_score = float(reply.split("Importance:")[1].strip())
        return sentiment_score, importance_score
    except openai.error.RateLimitError as e:
        print(f"Rate limit error: {e}. Retrying...")
        time.sleep(1)  # Wait for 1 second before retrying
        return get_scores(row)
    except Exception as e:
        print(f"Error processing row: {e}")
        return None, None

# Apply parallel processing
def process_dataset_in_parallel(df, num_workers=4):
    """
    Processes a dataset in parallel using the GPT scoring function.
    """
    with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
        results = list(executor.map(get_scores, [row for _, row in df.iterrows()]))
    
    # Add the results back to the DataFrame
    df['sentiment_score'], df['importance_score'] = zip(*results)
    return df

# Load your dataset
# Replace this with the actual loading of your dataset
# For example: df = pd.read_csv("your_dataset.csv")

df = d.sample(5)

# Process the dataset
d = process_dataset_in_parallel(d)

# Save the updated dataset
d.to_csv("scored_dataset.csv", index=False)

print("Processing complete. Scores added to dataset.")

Error processing row: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-znKNY5dUobluu5YtGTVoLdnz on tokens per min (TPM): Limit 200000, Used 199776, Requested 1712. Please try again in 446ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}
Error processing row: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-znKNY5dUobluu5YtGTVoLdnz on tokens per min (TPM): Limit 200000, Used 199289, Requested 1342. Please try again in 189ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}
Error processing row: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-znKNY5dUobluu5YtGTVoLdnz on tokens per min (TPM): Limit 200000, Used 198947, Requested 1855. Please try again in 240ms. Visit ht

KeyboardInterrupt: 

In [40]:
d.keys()

Index(['ticker', 'Date', 'category', 'title', 'content', 'Open', 'High', 'Low',
       'Close', 'Adj Close', 'Volume', 'label'],
      dtype='object')